<a href="https://colab.research.google.com/github/thomouvic/txtanalytics/blob/main/tfidf_cos_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget https://raw.githubusercontent.com/thomouvic/txtanalytics/main/docnames.csv
!wget https://github.com/thomouvic/txtanalytics/raw/main/docvecs.csv

--2022-11-14 00:28:11--  https://raw.githubusercontent.com/thomouvic/txtanalytics/main/docnames.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10660 (10K) [text/plain]
Saving to: ‘docnames.csv’

docnames.csv        100%[===================>]  10.41K  --.-KB/s    in 0s      

2022-11-14 00:28:11 (42.2 MB/s) - ‘docnames.csv’ saved [10660/10660]

--2022-11-14 00:28:11--  https://github.com/thomouvic/txtanalytics/raw/main/docvecs.csv
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/thomouvic/txtanalytics/main/docvecs.csv [following]
--2022-11-14 00:28:12--  https://raw.githubusercontent.com/th

In [4]:
import pandas as pd

df_docnames = pd.read_csv('docnames.csv')
# print(df_docnames)

df3 = pd.read_csv('docvecs.csv')
# print(df3)

# top k number of words to use for similarity
k = 100

# only consider the top k for each doc
df4 = df3.groupby(by='docid').head(k)
# print(df4) 

In [1]:
# A: # Given a docname find similar documents using cosine similarity on tfidf vectors
docname_given = 'txts/Arbuckle-Christie-Siemens_IntroductionSRC7.2_11-15-16.txt'

In [14]:
# Find docid for this docname_given.
docid = df_docnames[ df_docnames['docname']==docname_given ].iloc[0,1]
# print(docid)

# Find docvec for this docid.
docvec = df4[ df4['docid'] == docid ]
# print(docvec)

# Find the docvecs of docs that contain one of the terms in this docvec. 
# Only the rows of df4 that contain one of these terms are relevant for cosine similarity. 
# Then, remove rows with docid same as the document we are given 
# because we don't want to show the doc itself being similar to itself. 
relevant_docvecs = df4.merge(docvec[['termid', 'score']], on='termid') # this is an inner join 
relevant_docvecs = relevant_docvecs[ relevant_docvecs['docid'] != docid ]
relevant_docvecs = relevant_docvecs.sort_values(['docid'])
relevant_docvecs['scoremult'] = relevant_docvecs['score_x']*relevant_docvecs['score_y']
# print(relevant_docvecs)
simdocs = relevant_docvecs.groupby(by='docid')['scoremult'].sum().sort_values(ascending=False)
print(simdocs)

docid
7      0.351171
6      0.270747
5      0.184882
12     0.183604
99     0.150295
         ...   
73     0.001926
117    0.001669
63     0.001176
62     0.000933
0      0.000831
Name: scoremult, Length: 103, dtype: float64


In [20]:
# Retrieve the names of the top 5 most similar docs
df_simdocs = simdocs.to_frame()
df_simdocs['docid'] = df_simdocs.index
df_simdocs.reset_index(drop = True, inplace = True)
df_simdocs.head(5).merge(df_docnames, on='docid')

,scoremult,docid,docname
0,0.351171,7,txts/Arbuckle-Mauro-Siemens_Intro-Sydney-Whist...
1,0.270747,6,txts/Arbuckle-Crompton-Mauro_SRC-intro_Dec14.txt
2,0.184882,5,txts/Arbuckle-Christie_Intersections-Between_S...
3,0.183604,12,txts/Arbuckle-et-al-2020-Introduction-Open-Sch...
4,0.150295,99,txts/Siemens-2016-Faster-Alone-Further-Togethe...
